<a href="https://colab.research.google.com/github/SebastianLarssonDTU/02456-Reinforcement-Learning-Project/blob/before_framestacking/Generate_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INIT : Procgen, Drive, Git

In [1]:
!pip install procgen

In [2]:
#Clone git
!git clone -b before_framestacking https://github.com/SebastianLarssonDTU/02456-Reinforcement-Learning-Project.git "my_project"

fatal: destination path 'my_project' already exists and is not an empty directory.


In [3]:
#update git
%cd /content/my_project
! git pull

/content/my_project
Already up to date.


In [4]:
import datatools as tools
from datatools import DATA_PATH, MODEL_PATH
#Mount drive
tools.mount_drive()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# TESTING

In [5]:
#Import all custom files
import baseline
import datatools as tools
import hyperparameters as h
import model
import my_util
import policy
import ppo
import utils

#other imports
import torch

#import specific methods
from baseline import set_hyperparameters
from ppo import PPO
from experiments import run_experiment, print_list_of_experiments


In [6]:
# run_experiment(2, levels=50, save_interval=5e5)

In [7]:
# #for Experiment 7
# file_name = "Experiment7_50levels_Run_04Dec_18h12m45s_loaded_05Dec_10h49m07s_13516800steps"
# set_hyperparameters(baseline="Impala")
# h.value_clipping = True
# h.num_levels = 50
# h.batch_size=512
# model = PPO(print_output=True, eval=True, save_interval=5e5)
# model.load_policy(file_name)
# model.train()

In [8]:
# #For Experiment 2
# file_name = "Experiment2_50levels_Run_04Dec_18h10m32s_loaded_05Dec_09h43m36s_13516800steps"
# set_hyperparameters(baseline="PPO")
# h.value_clipping = True
# h.num_levels = 50
# model = PPO(print_output=True, eval=True, save_interval=5e5)
# model.load_policy(file_name)
# model.train()

In [9]:
# #for Experiment 8
# file_name = "Experiment8_50levels_Run_05Dec_08h34m02s_4005888steps"
# set_hyperparameters(baseline="Impala")
# h.value_clipping = True
# h.num_levels = 50
# h.batch_size=512
# h.death_penalty = True
# h.penalty = 5
# model = PPO(print_output=True, eval=True, save_interval=5e5)
# model.load_policy(file_name)
# model.train()

# Generate video

In [17]:
from utils import make_env
import imageio

def old_policy_evaluation(model, video_name=None, print_output=True, test_on_training=False, off_set = 0):
  policy = model.policy

  if video_name is None:
    video_name = model.file_name
  # Make evaluation environment
  if test_on_training:
    eval_env = make_env(model.num_envs, num_levels=model.num_levels)
    video_name += "_TEST_ON_TRAINING"
  else:
    eval_env = make_env(model.num_envs, start_level=model.num_levels+off_set, num_levels=model.num_levels)
  obs = eval_env.reset()

  frames = []
  total_reward = []

  # Evaluate policy
  policy.eval()
  # for _ in range(512):  #<--- Hardcoded batch size?
  while True:
    # Use policy
    action, log_prob, value = policy.act(obs)

    # Take step in environment
    obs, reward, done, info = eval_env.step(action)
    total_reward.append(torch.Tensor(reward))

    # Render environment and store
    frame = (torch.Tensor(eval_env.render(mode='rgb_array'))*255.).byte()
    frames.append(frame)
    if done[0]:
      break

  # Calculate average return
  total_reward = torch.stack(total_reward).sum(0).mean(0)
  
  if print_output:
    print('Average return:', total_reward)

  # Save frames as video
  frames = torch.stack(frames).cpu().numpy()
  imageio.mimsave(video_name+'.mp4', frames, fps=25)
  
  return total_reward

## Load model

In [11]:
# #for Experiment 8
# file_name = "Experiment8_200levels_Run_06Dec_16h17m59s_loaded_08Dec_10h26m35s_5021696steps"
# set_hyperparameters(baseline="Impala")
# h.value_clipping = True
# h.num_levels = 200
# h.batch_size=512
# h.death_penalty = True
# h.penalty = 5
# model = PPO(print_output=True, eval=True, save_interval=5e5)
# model.load_policy(file_name)

In [12]:
# old_policy_evaluation(model, test_on_training=True)

### Impala no penalty 200 levels

In [13]:
# #for Experiment 7
# file_name = "Experiment7_200levels_Run_06Dec_16h18m31s_loaded_07Dec_11h04m01s_7012352steps"
# set_hyperparameters(baseline="Impala")
# h.value_clipping = True
# h.num_levels = 200
# h.batch_size=512
# model = PPO(print_output=True, eval=True, save_interval=5e5)
# model.load_policy(file_name)

In [14]:
# old_policy_evaluation(model)

### PPO 200 levels

In [15]:
#For Experiment 2
file_name = "Experiment2_200levels_Run_07Dec_13h49m13s_loaded_08Dec_00h03m23s_7012352steps"
set_hyperparameters(baseline="PPO")
h.value_clipping = True
h.num_levels = 200
model = PPO(print_output=True, eval=True, save_interval=5e5)
policy=model.load_policy(file_name)

Observation space: Box(0.0, 1.0, (3, 64, 64), float32)
Action space: 15
Loaded current model from models folder with name Experiment2_200levels_Run_07Dec_13h49m13s_loaded_08Dec_00h03m23s_7012352steps.pt


In [18]:
for i in range(4):
  result=old_policy_evaluation(model, video_name="PPO_200levels_{}".format(i), off_set=i, print_output=False)
  print("{}: {}".format(i, result))
  

Average return: tensor(5.6914)
0: 5.691443920135498
Average return: tensor(6.4175)
1: 6.417477607727051
Average return: tensor(8.3341)
2: 8.334056854248047
Average return: tensor(10.9471)
3: 10.947111129760742


In [19]:
old_policy_evaluation(model, test_on_training=True)

Average return: tensor(13.1171)


tensor(13.1171)